# Homework 3 - What movie to watch tonight?

Goal of this homework is to build a search engine over a list of movies that have a dedicated page on Wikipedia. In order to achieve this goal, our work has been divided into different tasks that allowed us to fulfill the intent. Since we based the entire process upon using custom functions (choice due to the will of lightening the code and make it more readable), you won't find much commands, and we suggest to check the py files stored in this repository to better understand every step of which the notebook consists.
Let's start.

## Libraries

In [1]:
import os
import pickle
import pandas as pd
from tqdm import tqdm

## 1. Data collection

The first step to be taken, as a means to build the search engine, will be scraping the Wikipedia pages and store every information needed to build a dataframe from which our SE picks the useful ones and returns the results of a query. 

### 1.1. Get the list of movies

First thing first, we need the URLs. To acquire them, we created a function to download, directly from the page indicated by our TAs, the ones needed for the crawling phase.

In [2]:
import collector_utils

In [3]:
movies_urls = [collector_utils.get_urls(i) for i in range(1,3)]

In [4]:
dict(list(movies_urls[0].items())[0:10])

{1: 'https://en.wikipedia.org/wiki/Love_by_the_Light_of_the_Moon',
 2: 'https://en.wikipedia.org/wiki/The_Martyred_Presidents',
 3: 'https://en.wikipedia.org/wiki/Terrible_Teddy,_the_Grizzly_King',
 4: 'https://en.wikipedia.org/wiki/Jack_and_the_Beanstalk_(1902_film)',
 5: 'https://en.wikipedia.org/wiki/Alice_in_Wonderland_(1903_film)',
 6: 'https://en.wikipedia.org/wiki/The_Great_Train_Robbery_(1903_film)',
 7: 'https://en.wikipedia.org/wiki/The_Suburbanite',
 8: 'https://en.wikipedia.org/wiki/The_Little_Train_Robbery',
 9: 'https://en.wikipedia.org/wiki/The_Night_Before_Christmas_(1905_film)',
 10: 'https://en.wikipedia.org/wiki/Dream_of_a_Rarebit_Fiend_(1906_film)'}

In [5]:
dict(list(movies_urls[1].items())[0:10])

{10001: 'https://en.wikipedia.org/wiki/10_to_Midnight',
 10002: 'https://en.wikipedia.org/wiki/All_the_Right_Moves_(film)',
 10003: 'https://en.wikipedia.org/wiki/Americana_(film)',
 10004: 'https://en.wikipedia.org/wiki/Amityville_3-D',
 10005: 'https://en.wikipedia.org/wiki/Anna_to_the_Infinite_Power',
 10006: 'https://en.wikipedia.org/wiki/Baby_It%27s_You_(film)',
 10007: 'https://en.wikipedia.org/wiki/Bad_Boys_(1983_film)',
 10008: 'https://en.wikipedia.org/wiki/Better_Late_Than_Never_(film)',
 10009: 'https://en.wikipedia.org/wiki/The_Big_Chill_(film)',
 10010: 'https://en.wikipedia.org/wiki/The_Black_Stallion_Returns'}

As you can see, we generate two distinct dictionaries containing, for each list, a set of indexd URLs compliant to the original order.
Once yielded, we merge them and get the one that is going to be exploited for the latter purposes.

In [6]:
movies_urls[0].update(movies_urls[1])
movies_urls = movies_urls[0].copy()

### 1.2. Crawl Wikipedia

Here we start crawling the pages linked to the URLs. The custom function takes into account the possibility to get bloked by Wikipedia if too many requests are sent. To avoid this risk, we set a 20 minutes pause in case of [Response: 429]. Between any other two successful requests, instead, a random time lying within the range of 1-5 seconds is waited. If any other response code occurs, it just gets skipped and the loop continues (in this domain we find the case of non-existing pages). At the end of each iteration, an html file version of the page is stored in our laptop, within an 'Htmls' folder.

In [ ]:
collector_utils.scraping(movies_urls, 1)

### 1.3 Parse downloaded pages

Here we start parsing the htmls, that means: extracting from each one a list of specific informations that are going to be stored in a dataframe and used to retrieve the results after each research. In particular, we want to keep the following stuff:
<br>
**1. Title**
<br>
**2. Intro**
<br>
**3. Plot**
<br>
**4. Film name**
<br>
**5. Director**
<br>
**6. Producer**
<br>
**7. Writer**
<br>
**8. Starring**
<br>
**9. Music**
<br>
**10. Release date**
<br>
**11. Runtime**
<br>
**12. Country**
<br>
**13. Language**
<br>
**14. Budget**
<br>
Our final decision was, since not every Wiki page show the same template for the infoboxes dedicated to movies, to just exclude those ones that did not fit the scheme we built.

In [7]:
import parser_utils

In [ ]:
ls = ['Htmls/' + file for file in os.listdir('Htmls') if file[-4:] == 'html']
ls.sort(key = lambda x: int(''.join(filter(str.isdigit, x))))

In [ ]:
movie_data = list()
fails = 0

try:
    os.makedirs('Tsv')
except:
    _ = None

for i in tqdm(range(len(ls))):
    path_file = ls[i]
    movie_info = parser_utils.info_extractor(path_file)
    if movie_info != False:
        movie_data.append(movie_info)
        pd.DataFrame(list(movie_info.values())).to_csv(r'Tsv\Article_' + str(i + 1) + '.tsv', sep='\t', header = False, index = False)
    else:
        fails += 1
        #print(ls[i])

print(str(fails) + ' files are not Wikipedia movie pages')
movies_info_df = pd.DataFrame(movie_data)
movies_info_df['Doc_ID'] = movies_info_df.index
movies_info_df.to_csv('movie_data.csv')
movies_info_df = pd.read_csv('movie_data.csv')

As you can see, for each iteration a tsv file is created and then appended to a main dataframe which we will later use for our intentions.

## 2. Search Engine

Now that the preparatory phase has been brought to completion, we can start developing the main object of this homework: the search engine itself.
<br>
What we need to do, primarily, is running a preprocessing procedure. This will remove punctuation, stopwords and stem each document. We do not show it here, since our custom **text_cleaner** function is stored inside the **utils.py** file and is later used within the **index_utils.py** one, in which we decided to create a specific class of elements and methods to ease the work.

### 2.1. Conjunctive query

At this moment, we narrow our interest on the intro and plot of each document. It means that the first Search Engine will evaluate queries with respect to the aforementioned information.

#### 2.1.1 Create your index!

As already mentioned, the structure of our code was thought as finalized to get the py files recommended in the instructions, so it is not possible to split everything and show it in the notebook. Anyway, all that is needed to create the index is inside the **index_utils.py** library. What happens, basically, since we defined a class, is that, after initializing an _se_ object, we run a custom method (belonging to the class) called **create_engine**, and this one directly executes a series of commands that generate the index and the vocabulary for a specific search engine (which number will be taken as input, as well as the dataframe above which the procedures must be run).
<br>
The inverted index yielded will have the following shape:
<br>
$\{
term_id_1:[document_1, document_2, document_4],
\\ \ \ term_id_2:[document_1, document_3, document_5, document_6],\ ...\}$

In [8]:
import index_utils

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MOSES\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MOSES\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Loading the dataframe
movies_df = pd.read_csv('movie_data.csv')

In [10]:
# Initializing the object and running the method
se = index_utils.search_engine()
se.create_engine(movies_df)

In [ ]:
# Storing the resulting output
pickle.dump(se, open("se.p", "wb"))

#### 2.1.2 Execute the query

We can now execute a query over the built search engine. The output will be the documents containing all the words searched, of which we just show the title, the intro and the URL.

In [4]:
query = input()
se = pickle.load(open( "se.p", "rb" ))

disney movie 2019


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MOSES\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\MOSES\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [13]:
se.query(search_engine = 1, q = query, dataframe = movies_df)

,Title,Intro,Url
12875,Cars 3,\n\nCars 3 is a 2017 American 3D computer-anim...,https://en.wikipedia.org/wiki/Cars_3
11634,Frozen,\n\nFrozen is a 2013 American 3D computer-anim...,https://en.wikipedia.org/wiki/Frozen_(2013_film)
4211,Thumbelina,Thumbelina (also known as Hans Christian Ander...,https://en.wikipedia.org/wiki/Thumbelina_(1994...
11030,The Avengers,\n\nMarvel's The Avengers[6] (classified under...,https://en.wikipedia.org/wiki/The_Avengers_(20...
9658,The Chronicles of Narnia: Prince Caspian,\nThe Chronicles of Narnia: Prince Caspian is ...,https://en.wikipedia.org/wiki/The_Chronicles_o...


### 2.2 Conjunctive query & Ranking score

In this second search engine, given a query, we are assigned the task get the top-k documents related to the latter. 
In particular, we have to:
- find all the documents that contains all the words in the query.
- sort them by their similarity with the query
- return in output k documents, or all the documents with non-zero similarity with the query when the results are less than k. 

#### 2.2.1 Inverted index

Same old story: within the module **create_engine**, by means of an if condition, we specify the search engine number, so, when the search engine to produce is the second one, an inverted index based upon a _Tf-Idf_ score (computed real-time) gets created. The resulting shape will be the following: 
$ \{
term_id_1:[(document1, tfIdf_{term,document1}), (document2, tfIdf_{term,document2}), (document4, tfIdf_{term,document4}),\  ...],
\\
\ \ term_id_2:[(document1, tfIdf_{term,document1}), (document3, tfIdf_{term,document3}), (document5, tfIdf_{term,document5}), (document6, tfIdf_{term,document6}),\\ ...],\ ...\}$

In [ ]:
# Initializing the object and running the method. The second search engine is already trained via create_engnie() method.
se = index_utils.search_engine()
se.create_engine(movies_df)

#### 2.2.2 Execute the query

Besides the module to create a search engine with its corresponding index and vocabulary, our class has been furthermore provided of another one, called **query** (already used before), which directly computes the _Cosine similarity_ in case of search engines 2 & 3. In this way, when we browse it, our final output will be a list of documents, ranked by their Cosine similarity with respect to the query entered in input.

In [7]:
se.query(search_engine = 2, q = query, dataframe = movies_df)

,Title,Intro,Url,Score
4211,Thumbelina,Thumbelina (also known as Hans Christian Ander...,https://en.wikipedia.org/wiki/Thumbelina_(1994...,0.999152
11030,The Avengers,\n\nMarvel's The Avengers[6] (classified under...,https://en.wikipedia.org/wiki/The_Avengers_(20...,0.999003
12875,Cars 3,\n\nCars 3 is a 2017 American 3D computer-anim...,https://en.wikipedia.org/wiki/Cars_3,0.996418
9658,The Chronicles of Narnia: Prince Caspian,\nThe Chronicles of Narnia: Prince Caspian is ...,https://en.wikipedia.org/wiki/The_Chronicles_o...,0.987701
11634,Frozen,\n\nFrozen is a 2013 American 3D computer-anim...,https://en.wikipedia.org/wiki/Frozen_(2013_film),0.984607


## 3. Define a new score!

On this task we were asked to define a new score by which the results for a query could be ranked in a more efficacious way. Dealing with movies-centered data, it has been somewhat hard to come out with an idea. Our way to manage the job has consisted in suggesting the user a series of additional questions to guide him through the choice of more specific information. In particular, after computing the frequency by which a string appears inside different columns, the search engine recommends the user how to rank the data that most properly match its request. In this way a different weight (score = score * 1.02) will be assigned to the column to which we have been addressed.
<br>
Practical example: the user types "Brad Pitt", after retrieving info about movies that contains the specified string, the search engine will generate some questions like "Do you mean Directed by: brad?", "Do you mean Directed by: pitt?", "Do you mean Starring: brad?", "Do you mean Starring: pitt?" and "Do you mean Starring: brad pitt?". Of course, since we are dealing with conjunctive queries, the highest importance will be assigned to the results contemplating the whole string taken as input (in this case "brad pitt").

In [11]:
query = 'disney movie 2019'
se.query(search_engine = 3, q = query, dataframe = movies_df)

   frequency  length    word           col_name  Score
0          4       6  disney     Distributed by    4.6
1          3       6  disney  Productioncompany    3.9
do you mean Distributed by: disney? Y/n:y


,Title,Intro,Url,Score
11030,The Avengers,\n\nMarvel's The Avengers[6] (classified under...,https://en.wikipedia.org/wiki/The_Avengers_(20...,1.018983
12875,Cars 3,\n\nCars 3 is a 2017 American 3D computer-anim...,https://en.wikipedia.org/wiki/Cars_3,1.016346
9658,The Chronicles of Narnia: Prince Caspian,\nThe Chronicles of Narnia: Prince Caspian is ...,https://en.wikipedia.org/wiki/The_Chronicles_o...,1.007455
11634,Frozen,\n\nFrozen is a 2013 American 3D computer-anim...,https://en.wikipedia.org/wiki/Frozen_(2013_film),1.004300
4211,Thumbelina,Thumbelina (also known as Hans Christian Ander...,https://en.wikipedia.org/wiki/Thumbelina_(1994...,0.999152


In [10]:
query = 'Brad Pitt'
se.query(search_engine = 3, q = query, dataframe = movies_df)

   frequency  length       word     col_name  Score
0         31       9  brad pitt     Starring   24.4
1         32       4       brad     Starring   23.6
2         32       4       pitt     Starring   23.6
3          6       9  brad pitt  Produced by    6.9
4          6       4       brad  Produced by    5.4
5          6       4       pitt  Produced by    5.4
6          1       9  pitt brad     Starring    3.4
do you mean Starring: brad pitt? Y/n:y


,Title,Intro,Url,Score
10778,Happy Feet Two,\nHappy Feet Two (or Happy Feet 2) is a 2011 c...,https://en.wikipedia.org/wiki/Happy_Feet_Two,1.019958
8191,Troy,Troy is a 2004 epic historical war drama film ...,https://en.wikipedia.org/wiki/Troy_(film),1.019953
4032,Legends of the Fall,Legends of the Fall is a 1994 American epic hi...,https://en.wikipedia.org/wiki/Legends_of_the_Fall,1.019940
6726,Snatch,\nSnatch (stylized as snatch.) is a 2000 Briti...,https://en.wikipedia.org/wiki/Snatch_(film),1.019939
9197,The Assassination of Jesse James by the Coward...,The Assassination of Jesse James by the Coward...,https://en.wikipedia.org/wiki/The_Assassinatio...,1.019938
7107,Spy Game,Spy Game is a 2001 American spy film directed ...,https://en.wikipedia.org/wiki/Spy_Game,1.019936
12857,War Machine,War Machine is a 2017 American satirical war f...,https://en.wikipedia.org/wiki/War_Machine_(film),1.019928
8107,Ocean's Twelve,Ocean's Twelve is a 2004 American heist film d...,https://en.wikipedia.org/wiki/Ocean%27s_Twelve,1.019917
3209,Cool World,\nCool World is a 1992 American live-action/an...,https://en.wikipedia.org/wiki/Cool_World,1.019917
4610,Seven,\nSeven (stylized as SE7EN) is a 1995 American...,https://en.wikipedia.org/wiki/Seven_(1995_film),1.019908


## 4. Algorithmic question

In order to solve this problem we made multiple attempts, below we report all of them and explain the reasons behind every approach. Our first endeavour is actually very naive and to carry it out we decided to exploit itertools library. Our idea was: starting from the full sequence, if we set up a loop that decreases by one at each iteration,  we can compute all the combinations without replacement, and check if the resulting ones are equal to their reverse. If at least one of them satisfies this condition, the loop stops and we print the length of that which will be the longest palindromic sequence.

In [2]:
import time

In [27]:
from itertools import combinations

sequence = input('First algorithm implementation. Please insert a string: ')

start_time = time.time()

count = set()
done = False
for i in range(len(sequence), 1, -1):
    if (time.time() - start_time) > 10:
        break
    subsequences = list(combinations(sequence, i))
    for k in range(len(subsequences)):
        if subsequences[k] == subsequences[k][::-1]:
            count.add(len(subsequences[k]))
            done = True
            break
    if done: break

try:
    print('The longest palindromic subsequence has length: ' + str(max(count)))
except ValueError:
    print('There is no palindromic sequence or the elapsed time was higher than 10 seconds.')
    

elapsed_time = time.time() - start_time

print('The algorithm took approximately ' + str(round(elapsed_time, 2)) + ' seconds.')

First algorithm implementation. Please insert a string: dataminingsapienza2019902
It would take too much time.
There is no palindromic sequence or the elapsed time was igher than 10 seconds.
The algorithm took approximately 10.85 seconds.


The previous algorithm works, but it's obviously not optimal, because the time taken to run it increases as the sequence length gets longer. Hence we tried to propose an alternative based upon recursion.

In [41]:
def palindrome(sequence, i, j): 
    if (i == j): 
        return 1
    if (sequence[i] == sequence[j] and i + 1 == j): 
        return 2
    if (sequence[i] == sequence[j]): 
        return palindrome(sequence, i + 1, j - 1) + 2
    return max(palindrome(sequence, i, j - 1), palindrome(sequence, i + 1, j)) 

sequence = input('Second algorithm implementation. Please insert a string: ')
   
start_time = time.time()

length = len(sequence) 

if int(length) < 27:
    print("The longest palindromic subsequence has length: ", palindrome(sequence, 0, length - 1)) 
else:
    print('It would take too much time.')

elapsed_time = time.time() - start_time

print('The algorithm took approximately ' + str(round(elapsed_time, 2)) + ' seconds.')

Second algorithm implementation. Please insert a string: dataminingsapienza20199098
The longest palindromic subsequence has length:  7
The algorithm took approximately 17.03 seconds.


Even in this case, although an improvement in terms of space is obtained (and that is not trivial), the running time continues to be very long. Our last try is based upon Dynamic Programming, and we wrote the code by studying the algorithm derived from this approach. 

In [7]:
sequence = input('Third algorithm implementation. Please insert a string: ') 

start_time = time.time()

table = [[1 for x in range(len(sequence))] for x in range(len(sequence))] 

for i in range(len(sequence)): 
    table[i][i] = 1

for substring_length in range(2, len(sequence) + 1): 
    for i in range(len(sequence)-substring_length + 1): 
        j = i + substring_length-1
        if sequence[i] == sequence[j] and substring_length == 2: 
            table[i][j] = 2
        elif sequence[i] == sequence[j]: 
            table[i][j] = table[i + 1][j-1] + 2
        else: 
            table[i][j] = max(table[i][j-1], table[i + 1][j])

print("The longest palindromic subsequence has length: " + str(table[0][len(sequence)-1])) 

elapsed_time = time.time() - start_time

print('The algorithm took approximately ' + str(round(elapsed_time, 2)) + ' seconds.')

dataminingsapienzadataminingsapienza
The longest palindromic subsequence has length: 15
The algorithm took approximately 0.0 seconds.


This time the running time experience a substantial improvement and even with very long strings the result it's computed quickly and efficiently.